# Script for reading and plotting cle output of both OUT Stokes IQUV and ATMOS atmospheric parameters

### 1. Controling variables and imports 

In [13]:
import struct
import numpy as np
import os.path
import math
import re
import numba
from scipy import stats
from pylab import *
##file manipulation
from astropy.io import ascii
from astropy.io import fits
from scipy.io import FortranFile
import h5py
import pickle
##os and other stuff
import time
import warnings
import os
import sys
from numba import jit
#for ulterior plotting
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
plt.ion()

## to import the custom library in ipython
sys.path.insert(1, '/home/alin/Documents/physics_prog/cle/python/')
import importlib        ### to reload external files after changes are made within modules.
import cle_utils
import cle

## kr              transition index (the atom contains several radiative transitions, kr is in the order defined in the used ATOM file
##                 e.g. for Fe XIII kr=0 means 1074.7nm and kr=1 means 1079.8nm
##
## simpath_out     takes the path to the CLE OUT file.
## simpath_atmos   takes the path to the CLE OUT file.
##
## no_rot          is a binary argument used to not rotate q and u vectors from CLE.
##                 The CLE routine corona.f contains the definition of the reference direction.
##                 The Stokes is computed using a ref direction parallel to the z-axis [N-S] to a 
##                 polarization vector in the frame of (y,z) [E-W, N-S] in the plane of the sky. 
##                 The input numbers q,u are replaced with these components.       

kr_out=0
no_rot_out =1
simpath_out=    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_090_ls'
simpath_atmos=  '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/ATMOS'

# colorbar function to have nice colorbars in figures
def colorbar(mappable):
   from mpl_toolkits.axes_grid1 import make_axes_locatable
   import matplotlib.pyplot as plt
   last_axes = plt.gca()
   ax = mappable.axes
   fig = ax.figure
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="3.5%", pad=0.05)
   cbar = fig.colorbar(mappable, cax=cax)
   #cbar.formatter.set_powerlimits((4, 9))
   plt.sca(last_axes)
   return cbar

### 2. Class to read the OUT and ATMOS file

In [4]:
##put everything from OUT in a variable
s = cle_utils.out_params(simpath_out,kr_out,no_rot_out)


##put everything from ATMOS in a variable
atm=cle_utils.atmos_params(simpath_atmos)

<class 'int'>
Wavelength integrated Stokes were written
Length of buffer is 463026


In [5]:
# Integrated profiles and magnetic field components
# bx1=np.sum(atm.bx,axis=0)
# by1=np.sum(atm.by,axis=0)
# bz1=np.sum(atm.bz,axis=0)
# nne1=np.average(atm.nne[60:90,:,:],axis=0)
# te1=np.average(atm.te[60:90,:,:],axis=0)
# #te1=atm.te[75,:,:]
# v1=np.average(atm.v,axis=0)
# vt1=np.average(atm.vt[60:90,:,:],axis=0)

# btot=np.sqrt(bx1*bx1+by1*by1+bz1*bz1)
# # a=np.where(atm.te[75,:,:]==0)
# # fig, plots = plt.subplots(nrows=2, ncols=4, figsize=(11, 9))
# # plots[0,0].imshow(te1.T,extent=[0.8,1.5,-1.1,1.1])
# # c1 =patches.Circle((0, 0),1, color='red')
# # plots[0,0].add_patch(c1)
# %matplotlib widget

# fig, plots = plt.subplots(nrows=2, ncols=4, figsize=(11, 9))
# ab=plots[0,0].imshow(bx1.T/1000,extent=[0.8,1.5,-1.1,1.1])
# colorbar(ab)
# plots[0,0].set_title('B$_x$')
# plots[0,0].set_ylabel('Z [R$_\odot$]')
# plots[0,0].set_xlabel('Y [R$_\odot$]')
# c1 =patches.Circle((0, 0),1, color='black')
# plots[0,0].add_patch(c1)

# ab=plots[0,1].imshow(by1.T/1000,extent=[0.8,1.5,-1.1,1.1])
# plots[0,1].set_title('B$_y$')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[0,1].add_patch(c1)

# ab=plots[0,2].imshow(bz1.T/1000,extent=[0.8,1.5,-1.1,1.1])
# plots[0,2].set_title('B$_z$')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[0,2].add_patch(c1)

# ab=plots[0,3].imshow(btot.T/1000,extent=[0.8,1.5,-1.1,1.1])
# plots[0,3].set_title('B$_{tot}$')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[0,3].add_patch(c1)

# ab=plots[1,0].imshow(nne1.T/5,extent=[0.8,1.5,-1.1,1.1])
# plots[1,0].set_title('$\\rho$')
# colorbar(ab)
# plots[1,0].set_ylabel('Z [R$_\odot$]')
# plots[1,0].set_xlabel('Y [R$_\odot$]')
# c1 =patches.Circle((0, 0),1, color='black')
# plots[1,0].add_patch(c1)

# ab=plots[1,1].imshow(te1.T,extent=[0.8,1.5,-1.1,1.1])
# plots[1,1].set_title('T$_e$')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[1,1].add_patch(c1)

# ab=plots[1,2].imshow(v1.T,extent=[0.8,1.5,-1.1,1.1])
# plots[1,2].set_title('LOS Speed')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[1,2].add_patch(c1)

# ab=plots[1,3].imshow(vt1.T,extent=[0.8,1.5,-1.1,1.1])
# plots[1,3].set_title('Turb. speed')
# colorbar(ab)
# c1 =patches.Circle((0, 0),1, color='black')
# plots[1,3].add_patch(c1)

# plt.tight_layout()


In [6]:
# print(atm.bz.shape)
# fig = plt.figure()
# #plt.imshow(atm.nne[75,:,:].T)

# plt.plot(atm.nne[:,41,108])
# #print(atm.nne[:,41,108])

In [7]:
# from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.voxels(atm.nne)

In [8]:
##save atmos VTK datacubes for paraview rendering
#cle_utils.atmos2vtk(atm,"/home/alin/Documents/physics_prog/cle/python/","test_deg",bx=1,by=1,bz=1,nne=1,te=1)

## Plot and check IQUV and prepare for interpretation

In [9]:
### paths for the 3 cases of dipole; each for an ls scheme (2-1=1.5;) mix scheme (2-1=1.457); and extreme example (2-1=2.457)
simpath_out60_0  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_060_ls'
simpath_out60_1  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_060_mix'
simpath_out60_2  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_060_ext'
simpath_out90_0  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_090_ls'
simpath_out90_1  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_090_mix'
simpath_out90_2  =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_090_ext'
simpath_out125_0 =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_125_ls'
simpath_out125_1 =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_125_mix'
simpath_out125_2 =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_125_ext'
##si10
simpath_out90_si10 =    '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/OUT_001_095_090_090_si10'

In [10]:
s = cle_utils.out_params(simpath_out,kr_out,no_rot_out)

<class 'int'>
Wavelength integrated Stokes were written


In [11]:
##plot IQUV 90
# %matplotlib widget

# s = cle_utils.out_params(simpath_out90_0,kr_out,no_rot_out)
# fig, plots0 = plt.subplots(nrows=1, ncols=4, figsize=(11, 7))
# cd=plots0[0].imshow(s.data1.T,extent=[0.8,1.5,-1.1,1.1],vmin=-5,vmax=5,cmap="RdBu")
# colorbar(cd)
# c1 =patches.Circle((0, 0),1, color='black')
# plots0[0].add_patch(c1)
# plots0[0].set_title('Stokes I')

# ab=plots0[1].imshow(s.data4.T,extent=[0.8,1.5,-1.1,1.1],vmin=-0.15,vmax=0.15,cmap="RdBu")
# plots0[1].set_title('Stokes V')
# c1 =patches.Circle((0, 0),1, color='black')
# plots0[1].add_patch(c1)
# colorbar(ab)

# ab=plots0[2].imshow(s.data2.T,extent=[0.8,1.5,-1.1,1.1],vmin=-0.03,vmax=0.03,cmap="RdBu")
# plots0[2].set_title('Stokes Q')
# c1 =patches.Circle((0, 0),1, color='black')
# plots0[2].add_patch(c1)
# colorbar(ab)

# ab=plots0[3].imshow(s.data3.T,extent=[0.8,1.5,-1.1,1.1],vmin=-0.015,vmax=0.015,cmap="RdBu")
# colorbar(ab)
# plots0[3].set_title('Stokes U')
# c1 =patches.Circle((0, 0),1, color='black')
# plots0[3].add_patch(c1)


# plots0[0].set_ylabel('Z [R$_\odot$]')
# plots0[0].set_xlabel('Y [R$_\odot$]')
# #plt.title('LS Coupling')
# plt.tight_layout()


# s = cle_utils.out_params(simpath_out90_1,kr_out,no_rot_out)
# fig, plots1 = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
# plots1[0,0].imshow(s.data1.T,aspect=0.5)
# plots1[0,0].set_title('Stokes I')
# plots1[0,1].imshow(s.data4.T,aspect=0.5)
# plots1[0,1].set_title('Stokes V')

# plots1[1,0].imshow(s.data2.T,aspect=0.5)
# plots1[1,0].set_title('Stokes Q')
# plots1[1,1].imshow(s.data3.T,aspect=0.5)
# plots1[1,1].set_title('Stokes U')
# #plt.title('Mixed Coupling')
# plt.tight_layout()


# s = cle_utils.out_params(simpath_out90_2,kr_out,no_rot_out)
# fig, plots2 = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
# plots2[0,0].imshow(s.data1.T,aspect=0.5)
# plots2[0,0].set_title('Stokes I')
# plots2[0,1].imshow(s.data4.T,aspect=0.5)
# plots2[0,1].set_title('Stokes V')

# plots2[1,0].imshow(s.data2.T,aspect=0.5)
# plots2[1,0].set_title('Stokes Q')
# plots2[1,1].imshow(s.data3.T,aspect=0.5)
# plots2[1,1].set_title('Stokes U')
# #plt.title('Extreme 2-1 Lange G')
# plt.tight_layout()

In [12]:
##PLOT IQUV 125
# %matplotlib widget

# s = cle_utils.out_params(simpath_out125_0,kr_out,no_rot_out)
# fig, plots0 = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
# plots0[0,0].imshow(s.data1.T,aspect=0.5)
# plots0[0,0].set_title('Stokes I')
# plots0[0,1].imshow(s.data4.T,aspect=0.5)
# plots0[0,1].set_title('Stokes V')

# plots0[1,0].imshow(s.data2.T,aspect=0.5)
# plots0[1,0].set_title('Stokes Q')
# plots0[1,1].imshow(s.data3.T,aspect=0.5)
# plots0[1,1].set_title('Stokes U')

# #plt.title('LS Coupling')
# plt.tight_layout()


In [13]:
#plot IQUV 60

# %matplotlib widget

# s = cle_utils.out_params(simpath_out60_0,kr_out,no_rot_out)
# fig, plots0 = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
# plots0[0,0].imshow(s.data1.T,aspect=0.5)
# plots0[0,0].set_title('Stokes I')
# plots0[0,1].imshow(s.data4.T,aspect=0.5)
# plots0[0,1].set_title('Stokes V')

# plots0[1,0].imshow(s.data2.T,aspect=0.5)
# plots0[1,0].set_title('Stokes Q')
# plots0[1,1].imshow(s.data3.T,aspect=0.5)
# plots0[1,1].set_title('Stokes U')

# #plt.title('LS Coupling')
# plt.tight_layout()

In [14]:
##plot Stokes I for all 3

# %matplotlib widget

# fig, plots0 = plt.subplots(nrows=2, ncols=3, figsize=(8, 7))

# s = cle_utils.out_params(simpath_out60_0,kr_out,no_rot_out)
# ss1=s.data1.T
# plots0[0,0].imshow(s.data1.T,aspect=0.5,extent=[0.8,1.5,-1.1,1.1])
# plots0[0,0].set_title('Stokes I - $\Theta_{LOS}$= 60')
# plots0[0,0].set_ylabel('Z [R$_\odot$]')
# plots0[0,0].set_xlabel('Y [R$_\odot$]')
# c1 =patches.Circle((0, 0),1, color='red')
# plots0[0,0].add_patch(c1)

# s = cle_utils.out_params(simpath_out90_0,kr_out,no_rot_out)
# ss2=s.data1.T
# plots0[0,1].imshow(s.data1.T,aspect=0.5,extent=[0.8,1.5,-1.1,1.1])
# plots0[0,1].set_title('Stokes I - $\Theta_{LOS}$= 90')
# c1 =patches.Circle((0, 0),1, color='blue')
# plots0[0,1].add_patch(c1)


# s = cle_utils.out_params(simpath_out125_0,kr_out,no_rot_out)
# ss3=s.data1.T
# plots0[0,2].imshow(s.data1.T,aspect=0.5,extent=[0.8,1.5,-1.1,1.1])
# plots0[0,2].set_title('Stokes I - $\Theta_{LOS}$= 125')
# c1 =patches.Circle((0, 0),1, color='green')
# plots0[0,2].add_patch(c1)


# plots0[1,1].imshow((ss1+ss2+ss3)/3,aspect=0.5,extent=[0.8,1.5,-1.1,1.1])
# plots0[1,1].set_title('Stokes I - Combined')
# c1 =patches.Circle((0, 0),1, color='black')
# plots0[1,1].add_patch(c1)
# plots0[1,1].set_ylabel('Z [R$_\odot$]')
# plots0[1,1].set_xlabel('Y [R$_\odot$]')

# plots0[1,0].axis('off')
# plots0[1,2].axis('off')
# #plt.title('LS Coupling')
# plt.tight_layout()


In [15]:
##See where roughly the same pixel is in all 3 cases
# 90 case
# s = cle_utils.out_params(simpath_out90_0,kr_out,no_rot_out)
# fig=plt.figure()
# print(s.data1.shape)
# plt.plot(s.data1[:,108])
# print('max=',np.where(s.data1[:,108] == np.max(s.data1[:,108])))

In [16]:
##See where roughly the same pixel is in all 3 cases
# 60 case
# s = cle_utils.out_params(simpath_out60_0,kr_out,no_rot_out)
# fig=plt.figure()
# print(s.data1.shape)
# plt.plot(s.data1[:,108])
# print('max=',np.where(s.data1[:,108] == np.max(s.data1[:,108])))

In [17]:
##See where roughly the same pixel is in all 3 cases
# 125 case
# s = cle_utils.out_params(simpath_out125_0,kr_out,no_rot_out)
# fig=plt.figure()
# print(s.data1.shape)
# plt.plot(s.data1[:,108])
# print('max=',np.where(s.data1[:,108] == np.max(s.data1[:,108])))

In [18]:
#######LS Coupling

#for 90 deg pos; choosing pixel (108;41)
kr=0
s = cle_utils.out_params(simpath_out90_0,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])

kr=1
s = cle_utils.out_params(simpath_out90_0,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])
s_90_ls=np.concatenate((sa,sb))
##________________________________________________________________________________

#for 60 deg pos; choosing pixel (108;28)
kr=0
s = cle_utils.out_params(simpath_out60_0,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])

kr=1
s = cle_utils.out_params(simpath_out60_0,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])
s_60_ls=np.concatenate((sa,sb))
##________________________________________________________________________________

#for 125 deg pos; choosing pixel (108;24)
kr=0
s = cle_utils.out_params(simpath_out125_0,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])

kr=1
s = cle_utils.out_params(simpath_out125_0,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])
s_125_ls=np.concatenate((sa,sb))


<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193


In [19]:
## LC COUPLING FOR FeXIII and SiX line pair
## second for 90 deg pos (for using 2 atoms in LS); choosing pixel (108;41)
kr=0
s = cle_utils.out_params(simpath_out90_0,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])

s = cle_utils.out_params(simpath_out90_si10,kr,no_rot_out) ## ls 1430
print(s.wave)
sb=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])
s_90_2atoms=np.concatenate((sa,sb))
##________________________________________________________________________________


<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
14302.2314


In [20]:
#######Mixed  Coupling

#for 90 deg pos; choosing pixel (108;41)
kr=0
s = cle_utils.out_params(simpath_out90_1,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])

kr=1
s = cle_utils.out_params(simpath_out90_1,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])
s_90_mix=np.concatenate((sa,sb))
##________________________________________________________________________________


#for 60 deg pos; choosing pixel (108;28)
kr=0
s = cle_utils.out_params(simpath_out60_1,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])

kr=1
s = cle_utils.out_params(simpath_out60_1,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])
s_60_mix=np.concatenate((sa,sb))
##________________________________________________________________________________


#for 125 deg pos; choosing pixel (108;41)
kr=0
s = cle_utils.out_params(simpath_out125_1,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])

kr=1
s = cle_utils.out_params(simpath_out125_1,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])
s_125_mix=np.concatenate((sa,sb))


<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193


In [21]:
#######Exttreme 2->1 Lange G value (~2.5)  Coupling

#for 90 deg pos; choosing pixel (108;41)
kr=0
s = cle_utils.out_params(simpath_out90_2,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])

kr=1
s = cle_utils.out_params(simpath_out90_2,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[41,108],s.data2[41,108],s.data3[41,108],s.data4[41,108]])
s_90_ext=np.concatenate((sa,sb))
##________________________________________________________________________________


#for 60 deg pos; choosing pixel (108;28)
kr=0
s = cle_utils.out_params(simpath_out60_2,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])

kr=1
s = cle_utils.out_params(simpath_out60_2,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[28,108],s.data2[28,108],s.data3[28,108],s.data4[28,108]])
s_60_ext=np.concatenate((sa,sb))
##________________________________________________________________________________


#for 125 deg pos; choosing pixel (108;24)
kr=0
s = cle_utils.out_params(simpath_out125_2,kr,no_rot_out) ## ls 1074
print(s.wave)
sa=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])

kr=1
s = cle_utils.out_params(simpath_out125_2,kr,no_rot_out) ## ls 1079
print(s.wave)
sb=np.array([s.data1[24,108],s.data2[24,108],s.data3[24,108],s.data4[24,108]])
s_125_ext=np.concatenate((sa,sb))


<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193
<class 'int'>
Wavelength integrated Stokes were written
10746.1631
<class 'int'>
Wavelength integrated Stokes were written
10797.8193


In [22]:
print('LS Stokes vector 90:',s_90_ls)
print('LS Stokes vector 90 2atoms:',s_90_2atoms)
print('Mixed Stokes vector 90:',s_90_mix)
print('Large Lande Stokes vector 90:',s_90_ext)
print('LS Stokes vector 60:',s_60_ls)
print('Mixed Stokes vector 60:',s_60_mix)
print('Large Lande Stokes vector 90:',s_60_ext)
print('LS Stokes Vector 125:',s_125_ls)
print('Mixed Stokes Vector 125:',s_125_mix)
print('Large Lande Stokes vector 90:',s_125_ext)

LS Stokes vector 90: [ 2.788e+00 -2.410e-02  2.783e-03 -9.581e-04  2.110e+00 -5.816e-04
  6.708e-05 -7.508e-04]
LS Stokes vector 90 2atoms: [ 2.788e+00 -2.410e-02  2.783e-03 -9.581e-04  2.241e-01 -1.346e-03
  1.553e-04 -7.872e-05]
Mixed Stokes vector 90: [ 2.788e+00 -2.408e-02  2.781e-03 -9.576e-04  2.110e+00 -5.817e-04
  6.709e-05 -7.174e-04]
Large Lande Stokes vector 90: [ 2.788e+00 -2.408e-02  2.781e-03 -9.576e-04  2.110e+00 -5.817e-04
  6.709e-05 -1.467e-03]
LS Stokes vector 60: [ 2.663e+00 -2.277e-02  2.687e-03 -2.579e-03  1.948e+00 -5.431e-04
  6.407e-05 -2.233e-03]
Mixed Stokes vector 60: [ 2.663e+00 -2.276e-02  2.685e-03 -2.578e-03  1.948e+00 -5.432e-04
  6.408e-05 -2.133e-03]
Large Lande Stokes vector 90: [ 2.663e+00 -2.276e-02  2.685e-03 -2.578e-03  1.948e+00 -5.432e-04
  6.408e-05 -4.366e-03]
LS Stokes Vector 125: [ 3.074e+00 -1.670e-02  1.935e-03 -1.331e-03  2.299e+00 -4.266e-04
  4.935e-05 -9.260e-04]
Mixed Stokes Vector 125: [ 3.073e+00 -1.669e-02  1.933e-03 -1.331e-03  2

## Scaled down version of DBInvert to process these!

In [8]:
# We only use the Fe XIII combination or Fe XIII vs Si X here
## Only two line combinations are accepted

## Controling parameters
## number of lines to be used
nuse=2 
## switch for either two fe13 (nuse_comb=0) or fe13_1074+si10 (nuse_comb=1)
nuse_comb=0

#  the number of counts which correspond to the
#  largest intensity in each of the nobs observations
#  for example: counts = 1.e8 gives rms fluctuations 10^-4 * max(intensity)
counts=1.e10
## boolean variable to show the influences of photon count uncertainties on the fitting.
fiterr=1

## Number of db chi^2 fited solutions to presort.
## This is also the maximum number of solutions that are displayed/saved
partind=32

## How far from the best chi^2 fit to accept and print solutions
nind=10

# to make pdf files in directory "fitted" (can be time consuming)
iplot=0 



#### DB file directories for each Lande calculation

In [10]:
##Note:
## The Fe XIII atom calculations contain 2 lines, 
## The Si IX and Si X calculations contain 1 line each.


##standard db calculations:

#d_dir='/home/alin/Documents/physics_prog/cle/test_db_small/fe13/'   ##(the very compact collage2020 database)
#d_dir='/home/alin/Documents/physics_prog/cle/test_db_large'  ##(large database with mixed term atom)



## Specific degeneracy check db calculations
## height 1.001-1.489 R_sun
## dx=0.05 dy=0.02 dphi=2deg dtheta=2degs

#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_deg_fe13_ls/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_deg_fe13_mix/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_deg_fe13_ex/'


## Very fine db along y axis to check if problem is too ill-posed close to disk.
## heigth 1.001-1.256 R_sun
## dx=0.05 dy=0.003 dphi=2deg dtheta=2degs
########################CLE-DB 2.0.0##################################################
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db200_deg_121_fe13_ls/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db200_deg_121_fe13_mix/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db200_deg_121_fe13_ex/'
## also contains si10
#d_dir2='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db200_deg_121_si10_ls/'
########################CLE-DB 2.0.1##################################################
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db201_deg_121_fe13_ls/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db201_deg_121_fe13_mix/'
## list does not contain ex as it does not offer anything on top of mix
## also contains si10
#d_dir2='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db201_deg_121_si10_ls/'
#d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_201_fe13mix_multiB/' ## first multiB database
########################CLE-DB 2.0.2##################################################
d_dir='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/' # extreme lande database for the 3dipole simulation

#### Fake observation files

In [26]:
##rewrite the sobs array to a pixel of the user's choosing
##Not normed; remember to norm in the separate function!
    
#sobs1=s_90_ls
#sobs1=s_90_mix
#sobs1=s_90_ext

#sobs1=s_90_2atoms

#sobs1=s_60_ls
#sobs1=s_60_mix
#sobs1=s_60_ext

#sobs1=s_125_ls
#sobs1=s_125_mix
#sobs1=s_125_ext

##for the 3dipole simulation D1 and D2 points
sobs1=[ 2.77147984e+00, -1.30364644e-02, -8.49817523e-03, -5.08774743e-02, 1.90001667e+00, -3.30449505e-04, -2.13886508e-04, -7.47483540e-02] #D1
sobs1=[ 2.72472332e+00, -1.47195132e-04, -2.54103877e-05, -1.67518594e-01, 1.67537050e+00, -4.83628247e-06, -8.17960103e-07, -2.43508440e-01] #D2


In [27]:
## We need to define the observation apparent height
##from the x position of the pixels and the information in grid.dat
## 0.8 is the start value for the y grid, 70 is the length size for the y domain (nx= 150 along 1.5 r_sun --> ny=70 along 0.7r_sun)
yobs_0=0.8+(1.5-0.8)/70*41
yobs_1=0.8+(1.5-0.8)/70*28
yobs_2=0.8+(1.5-0.8)/70*24   

##yobs for 3dipole simulation
yobs_d1=1.11
yobs_d2=1.02

yobs=yobs_d2   ## choose based on situation for sobs ;0 is for 90, 1 is for 60 and 2 is for 125


print(yobs)

1.02


In [28]:
### test wether the database manipulation works. This vector should return entry 8553567 of the feXIII degeneracy(1.0-1.5 r_sun) database
#sobs=np.array([ 1.00000000e+00, -8.59024096e-02,  1.35502615e-02, -8.89804214e-03, 9.32414979e-02, -1.01014557e-03,  1.59340580e-04, -1.63879195e-03]) *( 1. + err*np.random.normal(0.,1,f.shape) )
#print(sobs)

In [29]:
# # ##process the Fe XIII and/or the Si X Databases 
# # # ## CLE-DB 2.0.0 

# importlib.reload(cle_utils) 
# if 1<= nuse <= 2:
#     g, ned, ngx, nbphi, nbtheta, xed, gxmin,gxmax, bphimin, bphimax,\
#     bthetamin, bthetamax, bfield, nline, nq = cle_utils.readhdr(d_dir+'db.hdr')
#     ## get the closest matching database entry
#     filen=cle_utils.get_db_entry(yobs,d_dir) 
#     file = open(d_dir+filen, 'rb')
#     print("height is: ",yobs,"\nDB height is: ",filen,file)

#     #
#     d = np.fromfile(file=file, dtype=np.float32)
#     shape=[ned*ngx*nbphi*nbtheta,nline,5]
#     data=np.reshape(d,shape)

#     # get stokes data s and alignments
#     #
#     #align=data[:,:,4]

#     # shape of the stokes data sdb ("s database")
#     #  is (number of calcs, 4*nuse)  
    
#     s0=data[:,0,:4]

#     if nuse == 1:   
#         sdb1=np.array((s0))
    
#     if nuse == 2:
#         if nuse_comb == 0:
#             s1=np.array((data[:,1,:4]))
        
#         elif nuse_comb == 1:
#             g, ned, ngx, nbphi, nbtheta, xed, gxmin,gxmax, bphimin, bphimax,\
#             bthetamin, bthetamax, bfield, nline, nq = cle_utils.readhdr(d_dir2+'db.hdr')
#             ## get the closest matching database entry
#             filen=cle_utils.get_db_entry(yobs,d_dir2) 
#             file = open(d_dir2+filen, 'rb')
#             print("height is: ",yobs,"\nDB height is: ",filen,file)
#             #
#             d = np.fromfile(file=file, dtype=np.float32)
#             shape=[ned*ngx*nbphi*nbtheta,nline,5]
#             data=np.reshape(d,shape)
#             s1=np.array((data[:,0,:4]))
    
#         else:
#             print("line combinations option not understood")
    
#         sdb1=np.append(s0,s1,axis=1)

# else:
#     print("Can only solve for one or two line combinations of observations")

# #print(sdb1[4422719,:],sdb1[4341571,:])
# print("selected database is of the form:", sdb1.shape)

In [30]:
#process the Fe XIII and/or the Si X Databases 
# CLE-DB 2.0.1 

# importlib.reload(cle_utils) 
# if 1<= nuse <= 2:
#     g, ned, ngx, nbphi, nbtheta, nb, xed, gxmin,gxmax, bphimin, bphimax,\
#     bthetamin, bthetamax, bfield, nline, nq = cle_utils.readhdr201(d_dir+'db.hdr')
#     ## get the closest matching database entry
#     filen=cle_utils.get_db_entry(yobs,d_dir) 
#     file = open(d_dir+filen, 'rb')
#     print("height is: ",yobs,"\nDB height is: ",filen,file,nb)

#     #
    
#     shap=[4*nline,ned*ngx*nbphi*nbtheta*nb]
#     s=np.empty(shap)
#     #
#     d = np.fromfile(file=file, dtype=np.float32)
#     shape=[ned*ngx*nbphi*nbtheta*nb,nline,5]
#     data=np.reshape(d,shape)

#     # get stokes data s and alignments
#     #
#     #align=data[:,:,4]

#     # shape of the stokes data sdb ("s database")
#     #  is (number of calcs, 4*nuse)  
    
#     s0=data[:,0,:4]

#     if nuse == 1:   
#         sdb1=np.array((s0))
    
#     if nuse == 2:
#         if nuse_comb == 0:
#             s1=np.array((data[:,1,:4]))
        
#         elif nuse_comb == 1:
#             g, ned, ngx, nbphi, nbtheta, nb, xed, gxmin,gxmax, bphimin, bphimax,\
#             bthetamin, bthetamax, bfield, nline, nq = cle_utils.readhdr201(d_dir2+'db.hdr')
#             ## get the closest matching database entry
#             filen=cle_utils.get_db_entry(yobs,d_dir2) 
#             file = open(d_dir2+filen, 'rb')
#             print("height is: ",yobs,"\nDB height is: ",filen,file)
#             #
#             d = np.fromfile(file=file, dtype=np.float32)
#             shape=[ned*ngx*nbphi*nbtheta,nline,5]
#             data=np.reshape(d,shape)
#             s1=np.array((data[:,0,:4]))
    
#         else:
#             print("line combinations option not understood")
    
#         sdb1=np.append(s0,s1,axis=1)

# else:
#     print("Can only solve for one or two line combinations of observations")

# #print(sdb1[4422719,:],sdb1[4341571,:])
# print("selected database is of the form:", sdb1.shape)

In [31]:
# ##process the Fe XIII and/or the Si X Databases 
# # ## CLE-DB 2.0.2 

importlib.reload(cle) 
if 1<= nuse <= 2:
    g=cle.readhdr(d_dir+'db.hdr')
    g, ned, ngx, nbphi, nbtheta, xed, gxmin,gxmax, bphimin, bphimax, \
    bthetamin, bthetamax, nline, nq  = cle.parsehdr(g)
    ## get the closest matching database entry
    filen=cle_utils.get_db_entry(yobs,d_dir) 
    file = open(d_dir+filen, 'rb')
    print("height is: ",yobs,"\nDB height is: ",filen,file)

    #
    d=cle.dcompress(np.fromfile(file=file, dtype=np.int16))
    shape=[ned*ngx*nbphi*nbtheta,nline,4]
    data=np.reshape(d,shape)
    
    s0=data[:,0,:4]

    if nuse == 1:   
        sdb1=np.array((s0))
    
    if nuse == 2:
        if nuse_comb == 0:
            s1=np.array((data[:,1,:4]))
        
        elif nuse_comb == 1:
            g, ned, ngx, nbphi, nbtheta, xed, gxmin,gxmax, bphimin, bphimax,\
            bthetamin, bthetamax, bfield, nline, nq = cle_utils.readhdr(d_dir2+'db.hdr')
            ## get the closest matching database entry
            filen=cle_utils.get_db_entry(yobs,d_dir2) 
            file = open(d_dir2+filen, 'rb')
            print("height is: ",yobs,"\nDB height is: ",filen,file)
            #
            d = np.fromfile(file=file, dtype=np.float32)
            shape=[ned*ngx*nbphi*nbtheta,nline,5]
            data=np.reshape(d,shape)
            s1=np.array((data[:,0,:4]))
    
        else:
            print("line combinations option not understood")
    
        sdb1=np.append(s0,s1,axis=1)

else:
    print("Can only solve for one or two line combinations of observations")

#print(sdb1[4422719,:],sdb1[4341571,:])
print("selected database is of the form:", sdb1.shape)

height is:  1.02 
DB height is:  DB0020.DAT <_io.BufferedReader name='/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/DB0020.DAT'>
155 MB in database file
DCOMPRESS: 0.269  SECONDS FOR WHERE
DCOMPRESS: 0.733  SECONDS FOR EXP
DCOMPRESS: 1.002  SECONDS TOTAL
selected database is of the form: (9720000, 8)


In [32]:
### scale both the database and observation 

#
# Now, re-scale qu and v by fq and fv to make s roughly (1,1,1,1)
# accordingly, the noise magnitudes must be scaled by the same factors:
#  Note: this tends to change only the chi2 values by a single multiplier
#  thereby not affecting the ordering of the chi2

#sobs1=sdb1[302,:]                  ####fake an observation
importlib.reload(cle_utils)       ### to parse changes on the imported module
sobs,sdb,rms = cle_utils.scale_stokes(sobs1,sdb1,nuse,counts)


#print(sobs1)
#print(sobs)
#print(sdb)
#print(np.max(sdb1[:,3]/sdb1[:,0]),np.min(sdb1[:,3]/sdb1[:,0]))

#print(0.99**2+0.3**2+0.09**2+0.0001**2+0.48**2+0.1**2+0.004**2+0.00005**2,0.000098)
#print(sdb1[295187,:])
#print(rms) 
# plt.figure()
# plt.plot(sdb1[:,5])
# plt.plot(sdb1[:,6])
# [ 6.6671162e-09 -9.2859653e-14 -1.1470989e-13 -5.9653501e-14 3.5679473e-09 -1.4855947e-14 -1.8351609e-14 -3.2077880e-14]LS
#[ 6.6673191e-09 -9.2810681e-14 -1.1464938e-13 -5.9742684e-14 3.5680743e-09 -1.4859441e-14 -1.8355921e-14 -3.0707174e-14] MIX
#print(-5.9653501e-14/6.6671162e-09, -3.2077880e-14/3.5679473e-09)
#print(-5.9742684e-14/6.6673191e-09,-3.0707174e-14/3.5680743e-09)

In [33]:
## statistics of chi^2 value versus the photon counts used.
# countchi=np.zeros(14)
# countchi[0] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1954941,:]/np.max(sdb1[1954941,:]))**2)/(sobs1.shape[0])   ##counts e^1
# countchi[1] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1204953,:]/np.max(sdb1[1204953,:]))**2)/(sobs1.shape[0])  #1204953
# countchi[2] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1946747,:]/np.max(sdb1[1946747,:]))**2)/(sobs1.shape[0])
# countchi[3] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1063755,:]/np.max(sdb1[1063755,:]))**2)/(sobs1.shape[0])
# countchi[4] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[15810,:]/np.max(sdb1[15810,:]))**2)/(sobs1.shape[0])
# countchi[5] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[9436,:]/np.max(sdb1[9436,:]))**2)/(sobs1.shape[0])        ##counts e^6
# countchi[6] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1052689,:]/np.max(sdb1[1052689,:]))**2)/(sobs1.shape[0])
# countchi[7] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[1879424,:]/np.max(sdb1[1879424,:]))**2)/(sobs1.shape[0])
# countchi[8] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[393,:]/np.max(sdb1[393,:]))**2)/(sobs1.shape[0])
# countchi[9] =np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[301,:]/np.max(sdb1[301,:]))**2)/(sobs1.shape[0])     ##counts e^10
# countchi[10]=np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[302,:]/np.max(sdb1[302,:]))**2)/(sobs1.shape[0])+0.3e-10
# countchi[11]=np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[302,:]/np.max(sdb1[302,:]))**2)/(sobs1.shape[0])+0.3e-10
# countchi[12]=np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[302,:]/np.max(sdb1[302,:]))**2)/(sobs1.shape[0])+0.3e-10
# countchi[13]=np.sum( (sdb1[302,:]/np.max(sdb1[302,:])-sdb1[302,:]/np.max(sdb1[302,:]))**2)/(sobs1.shape[0])+0.3e-10   ##counts e^14

# plt.figure()
# cou=np.array(([1e1,1e2,1e3,1e4,1e5,1e6,1e7,1e8,1e9,1e10,1e11,1e12,1e13,1e14]))
# plt.plot(cou,np.log(countchi))
# plt.yscale('symlog')
# plt.xscale('log')
# plt.xlabel("Counts")
# plt.ylabel("Log $\chi^2$")
# #plt.ylim([-1e-19,1e-3])
# print(countchi)

In [34]:
## check the magnetic field strength in the database and obs
## see the additional notebook that checks this!

In [35]:
##fit the database to your Stokes FeXIII pair
start=time.time()

#
nc=sobs.shape[0]
x=sobs

#1.    (Im - Io)^2 + (Qm - Qo)^2 + (Um - Uo)^2 +(Vm - Vo)^2
dif=(sdb-x)
if fiterr == 0:   ##don't include the rms in the chi2 fitting
    sdif=dif*dif
else:
    sdif=dif*dif/rms/rms
  
chi2 = np.sum(sdif, axis=1)/(nc-1)
print(dif.shape,sdif.shape)

#  Here is a minimal search algorithm- a mere partition and then sorting of chi2
#
ind = np.argpartition(chi2, partind)[:partind]
asrt= ind[np.argsort(chi2[ind])]
#


firstchi2=chi2[asrt[0]]
si=0
ix=asrt[si]
print("best s:\n",ix,sdb[ix,:],"\n",asrt[1],sdb[asrt[1],:])

#print("other s",sdb_a[512,ix,:])
print("OBS s:\n",x)
dt= "{:3.2f}".format(time.time()-start)
print("Calculation time:",dt,' seconds')


(9720000, 8) (9720000, 8)
best s:
 5945174 [ 1.0000e+00 -4.3276e+00  3.2557e+01 -9.7797e-04  6.1448e-01 -4.8542e-01
  3.6519e+00 -1.1798e-03] 
 6707116 [ 1.0000e+00 -4.3276e+00  3.2557e+01 -9.7797e-04  6.1448e-01 -4.8542e-01
  3.6519e+00 -1.1798e-03]
OBS s:
 [ 1.     -0.5835  0.15   -0.6148  0.6149 -0.214   2.5469 -0.8938]
Calculation time: 0.75  seconds


In [139]:

# fig, plots1 = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
# plots1[0].plot(chi2)
# plots1[0].set_yscale('log')
# plots1[1].hist(chi2,bins=1000);

In [94]:
# plt.figure()
# plt.plot(chi2)
# #plt.ylim([1e-2,4e4])
# #plt.ylim([1e2,4e10])
# # plt.yscale('log')# 
# plt.xlabel("Database entry")
# plt.ylabel("Log $\chi^2$")
# plt.annotate("Fe XIII 1074nm and 1079nm line pair",xy=[1e5,2e4])
# plt.annotate("Mixed LS+jj coupling scheme",xy=[1e5,1e4])
# plt.annotate("Photon counts ~10$^{11}$",xy=[1e5,0.5e4])
# plt.tick_params(axis="both",direction="in",which="both")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
outdir='../test_cle_degeneracy/fitted/'
oindex1=0
os.system("rm "+outdir+str(oindex1)+".txt")
Fobj = open(outdir+str(oindex1)+".txt","a")
Fobj.write("      O      C     chi2         Ne         y        x       phi      theta       |B| POS AZ  LOS THETA")
Fobj.write("\n")
print("      O       C      chi2         Ne        y         x       phi      theta       |B|     POS AZ   LOS THETA")

if(iplot > 0): 
    fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(10,10))
    lab=['I','Q','U','V']
    lab+=lab
    lab+=lab


si=0
ix=asrt[si]

s2d=180./np.pi

while ((chi2[ix]/firstchi2) < nind and (si <= partind-1)):
    ix=asrt[si]
    dd=chi2[ix]
    ddi= "{:0.2e}".format(dd)
    if(dd/firstchi2 < nind):
        np.savetxt(Fobj,[oindex1,ix], fmt='%7i', newline=" ")
        np.savetxt(Fobj,[chi2[ix]], fmt='%+.2e', newline=" ")
        np.savetxt(Fobj,[cle.physa(ix,yobs,g,1)],fmt='%+9.2f', newline=" ")
        Fobj.write("\n")
        a1,a2,a3,a4,a5,a6=cle.physa(ix,yobs,g,1)
        print('%7i  %7i  '%(oindex1,ix),end=" ")
        print('%+.2e'%chi2[ix],end="")
        print('%+9.2f %+9.2f %+9.2f %+9.2f %+9.2f %+9.2f'%(a1,a2,a3,a4,a5,a6),end="")
        los=np.arccos(np.sin(a5)*np.cos(a4))*s2d
        pos=np.arctan2(np.sin(a5)*np.sin(a4),np.cos(a5))*s2d
        print('%+9.2f %+9.2f'%(pos,los))
        #np.savetxt(Fobj, [pos,los],fmt='%+7.2f', newline=" ")
    if(iplot > 0):
        text="fit="+str(ix)+" $\chi^2$="+ddi
        if(si ==0):
            text="Obs sp="+str(oindex1)
            x1=np.copy(x)
            if (sdb.shape[1] >8 ): 
                x1[4:8]=x1[4:8]*mx1/(x1[4]*mx1) 
                x1[8:12]=x1[8:12]*mx2/(x1[8]*mx2)
            ax1.plot(x1,'.',label=text)
        if(si == 1):   
            text= "fit="+str(ix)+" $\chi^2$="+ddi
            sdb1=np.copy(sdb[ix,:]) 
            if (sdb.shape[1] >8 ):        
                sdb1[4:8]=sdb1[4:8]*mx1/(sdb1[4]*mx1) 
                sdb1[8:12]=sdb1[8:12]*mx2/(sdb1[8]*mx2)
            ax1.plot(sdb1,'.',label=text)
            ax1.errorbar(np.arange(nc),x,yerr=rms,fmt='none')
            ax1.legend(fontsize=7)
            ax1.set_xticks(np.arange(sdb.shape[1]))
            ax1.set_xticklabels(lab[:sdb.shape[1]])
            ax1.text(x=0.5,y=0.9,s="Fe ${XIII}$ 1074.7nm",size=7.5)
            if (sdb.shape[1] >4 ):
                ax1.axvline( x=3.5, ymin=-0.25, ymax=1)
                ax1.text(x=4.5,y=0.9,s="Fe ${XIII}$ 1079.8nm",size=7.5)
            if (sdb.shape[1] >8 ):
                ax1.axvline( x=7.5, ymin=-0.25, ymax=1)
                ax1.text(x=8.5,y=0.9,s="Si ${IX}$ 3934.3nm",size=7.5)
        if(si ==0): 
            ax2.plot(dif[ix,:],'g.-',label="(O-N)/$\sigma$")
            ax2.legend(fontsize=7)
            ax2.set_xticks(np.arange(sdb.shape[1]))
            ax2.set_xticklabels(lab)
            ax2.text(x=0.5,y=0.9,s="Fe ${XIII}$ 1074.7nm",size=7.5)   
            if (sdb.shape[1] >4 ):
                ax2.axvline( x=3.5, ymin=-0.25, ymax=1)
                ax2.text(x=4.5,y=0.9,s="Fe ${XIII}$ 1079.8nm",size=7.5)
            if (sdb.shape[1] >8 ):
                ax2.axvline( x=7.5, ymin=-0.25, ymax=1)
                ax2.text(x=8.5,y=0.9,s="Si ${IX}$ 3934.3nm",size=7.5)
    si=si+1
    

# a1,a2,a3,a4,a5,a6=physa(oindex1,yobs,g1)     
# print('\x1b[31mObservation Physics:        %+9.2f %+9.2f %+9.2f %+9.2f %+9.2f %+9.2f\x1b[0m'%(a1,a2,a3,a4,a5,a6))   
# Fobj.write("Observation: \n")         
# np.savetxt(Fobj,[0000,0000], fmt='%7i', newline=" ")
# np.savetxt(Fobj,[0000], fmt='%7i', newline=" ")
# np.savetxt(Fobj,[physa(oindex1,yobs,g1) ],fmt='%+9.2f', newline=" ")
# Fobj.write("\n")
# if(iplot > 0):
#     filen=outdir+'fit'+str(oindex1)
#     print("Saving ", filen+".pdf")
#     savefig(filen+".pdf")
#     #plt.close()
#
Fobj.close()

      O       C      chi2         Ne        y         x       phi      theta       |B|     POS AZ   LOS THETA
      0  5945174   +1.68e+07    +8.11     +1.02     -1.20     +6.21     +1.55     +1.00   -75.88     +4.15
      0  6707116   +1.68e+07    +8.05     +1.02     +1.25     +0.11     +1.62     +1.00  +116.76     +6.75
      0  6707114   +1.68e+07    +8.05     +1.02     +1.25     +0.11     +1.55     +1.00   +80.47     +6.12
      0  5945176   +1.68e+07    +8.11     +1.02     -1.20     +6.21     +1.62     +1.00  -127.07     +5.04
      0  6301845   +1.68e+07    +9.60     +1.02     -0.03     +0.00     +1.59     +1.00  +180.00     +1.01
      0  6334245   +1.68e+07    +9.59     +1.02     +0.08     +0.00     +1.59     +1.00  +180.00     +1.01
      0  6301844   +1.68e+07    +9.60     +1.02     -0.03     +0.00     +1.55     +1.00    +0.00     +1.01
      0  6334244   +1.68e+07    +9.59     +1.02     +0.08     +0.00     +1.55     +1.00    +0.00     +1.01
      0  6334246   +1.68e+07    +9

### Read an atmos file from DB

This has the purpose to understand what a 1G field strength pixel means in terms of CLE
Assuming the bx by and bz components are in maxwell units, e.g. 1000 ~1G

In [35]:
##put everything from ATMOS in a variable
atm=cle_utils.atmos_params('/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_test_ATMOS/ATMOS')

print(np.shape(atm.bx))


#Integrated profiles and magnetic field components
bx1=np.sum(atm.bx,axis=0)
by1=np.sum(atm.by,axis=0)
bz1=np.sum(atm.bz,axis=0)
nne1=np.sum(atm.nne,axis=0)
te1=np.average(atm.te,axis=0)
v1=np.average(atm.v,axis=0)
vt1=np.average(atm.vt,axis=0)

btot=np.sqrt(bx1*bx1+by1*by1+bz1*bz1)

Length of buffer is 302474
(15, 24, 15)


In [36]:
plt.figure()
plt.imshow(btot.T,aspect=0.5)
#plt.imshow(vt1,aspect=0.5)
# for i in range(0,32):
#     for j in range(0,14):
#         for k in range(0,6):
#             if (atm.bx[i,j,k] != 0):
#                 print(i,j,k)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
### s = cle_utils.out_params('/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db_test_ATMOS/OUT',kr_out,no_rot_out)
plt.figure()
plt.imshow(s.data4.T,aspect=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

###We now need to address the field scaling to compare with the database.

In [38]:
## the 4 pixels that correspond to a 1G field!
print(np.shape(btot))
print(np.sum(btot[18:20,4:6])/4)

(24, 15)
1006.5749575302418


In [39]:
## lets compute the b_los from the Stokes information

In [40]:

pc= 6.6261e-27 ##plank  cm^2 g s^-1
bm=9.274e-21 ##bohr magneton erg G^-1        
cc=2.99792458e10 #cm s-1 
me= 9.1095e-28   #g
ee=4.8032e-10    #esu  g1/2 cm3/2 s−1
kb=1.38064852e-16  #erg/K
te= 10**6.22
mion=55.845*1.672621E-24  #  g    for FeXIII 
omegat=math.sqrt(2*kb*te/mion)     #thermal velocity without microturbulence



landeg = 1.5
alamb  = 10746.8
#dlmd_d = omegat*alamb*1e-8/cc               #Doppler width in frequency units!   1e-8 -->conversion A to cm
dlmd_d = omegat/(1e-8* alamb)               #Doppler width in wavelength units!  1e-8 -->conversion A to cm
SV     = sdb2[7]
SI     = sdb2[4]
SQ     = sdb2[5]
SU     = sdb2[6]
L      = np.sqrt(SQ**2+SU**2)
F      = 0
sin2theta = 0

 ## only needed if you use different atom combinations
F1      = 0.0   ## Fe XIII any line
F2      = 0.5   ## Si X ; Dima & Schad 2020
SV1     = sdb2[3]
SV2     = sdb2[7]
SI1     = sdb2[0]
SI2     = sdb2[4]
SQ1     = sdb2[1]
SQ2     = sdb2[5]
SU1     = sdb2[2]
SU2     = sdb2[6]
L1      = np.sqrt(SQ1**2+SU1**2)
L2      = np.sqrt(SQ2**2+SU2**2)
landeg1 = 1.5
landeg2 = 1.5
sin2theta=(2/3)*( (F1*L1*SV2) - (F2*L2*SV1) )/ (landeg2*SV1*(SI2+L2) - landeg1*SV2*(SI2+L2))
##-----------------------------------------------------

blos_d=(pc/bm) * (-SV) / ((SI+L)*landeg + (2/3) * F2 * (L/sin2theta))  # eq 14 Dima & Schad 2020 LOS magnetic field.; CGS
blos_p= -8*math.pi*me*cc*SV*dlmd_d/(3*(SI+L)*ee)   #eq 14 Plowman, 2014: LOS magnetic field |B|cos(theta); units are cgs 

print(sin2theta)
print('dima :',blos_d)
print('plowman:',blos_p)

NameError: name 'sdb2' is not defined

In [ ]:
plt.figure()
#plt.plot(sdb[:,0]/sdb[:,3])
print(sobs[0]/sobs[3])